In [1]:
import pandas as pd
import numpy as np
import pickle

# Data Exploration

In [2]:
postings = pd.read_csv('/kaggle/input/linkedin-job-postings/postings.csv')
postings.sample(4)

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
43350,3900075341,JTC Group,Associate Director - Fund Services,EMPLOYMENT TYPE: At Will\n\n DEPARTMENT: Fun...,NaN,NaN,"Missouri, United States",1025207.0,4.0,NaN,...,NaN,1.713210e+12,jtcgroup.csod.com,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN
122167,3906252226,"PLS Financial Services, Inc.",Customer Service Representative- Bilingual Spa...,Overview\n\n PLS ® : People. Location. Service...,NaN,NaN,"Fort Worth, TX",23929.0,2.0,NaN,...,NaN,1.713566e+12,careers-pls.icims.com,0,FULL_TIME,NaN,NaN,NaN,76102.0,48439.0
51208,3901379605,Long Finch Technologies LLC,SAP ABAP Developer,Role: SAP ABAP Developer on Hana\n• Hands on S...,NaN,NaN,"Plano, TX",11187415.0,7.0,NaN,...,NaN,1.713460e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,75023.0,48085.0
24806,3891063748,Photon,BA - SME investment,Client: PhotonPosition: BA - Alternative Inves...,NaN,NaN,"Dallas, TX",165464.0,3.0,NaN,...,NaN,1.712860e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,75201.0,48113.0


In [3]:
postings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [4]:
col_dict = {
    "Application Types" : 'application_type',
    "Currencies" : 'currency',
    "Work Types": 'work_type',
    'Compensation Types' : 'compensation_type'
            }

for text, col in col_dict.items():
    print(text, ':', list(postings[col].unique()))


Application Types : ['ComplexOnsiteApply', 'SimpleOnsiteApply', 'OffsiteApply', 'UnknownApply']
Currencies : ['USD', nan, 'CAD', 'BBD', 'EUR', 'AUD', 'GBP']
Work Types : ['FULL_TIME', 'INTERNSHIP', 'CONTRACT', 'PART_TIME', 'TEMPORARY', 'VOLUNTEER', 'OTHER']
Compensation Types : ['BASE_SALARY', nan]


In [5]:
postings.describe()

,job_id,max_salary,company_id,views,med_salary,min_salary,applies,original_listed_time,remote_allowed,expiry,closed_time,listed_time,sponsored,normalized_salary,zip_code,fips
count,1.238490e+05,2.979300e+04,1.221320e+05,122160.000000,6280.000000,2.979300e+04,23320.000000,1.238490e+05,15246.0,1.238490e+05,1.073000e+03,1.238490e+05,123849.0,3.607300e+04,102977.000000,96434.000000
mean,3.896402e+09,9.193942e+04,1.220401e+07,14.618247,22015.619876,6.491085e+04,10.591981,1.713152e+12,1.0,1.716213e+12,1.712928e+12,1.713204e+12,0.0,2.053270e+05,50400.491887,28713.879887
std,8.404355e+07,7.011101e+05,2.554143e+07,85.903598,52255.873846,4.959738e+05,29.047395,4.848209e+08,0.0,2.321394e+09,3.622893e+08,3.989122e+08,0.0,5.097627e+06,30252.232515,16015.929825
min,9.217160e+05,1.000000e+00,1.009000e+03,1.000000,0.000000,1.000000e+00,1.000000,1.701811e+12,1.0,1.712903e+12,1.712346e+12,1.711317e+12,0.0,0.000000e+00,1001.000000,1003.000000
25%,3.894587e+09,4.828000e+01,1.435200e+04,3.000000,18.940000,3.700000e+01,1.000000,1.712863e+12,1.0,1.715481e+12,1.712670e+12,1.712886e+12,0.0,5.200000e+04,24112.000000,13121.000000
50%,3.901998e+09,8.000000e+04,2.269650e+05,4.000000,25.500000,6.000000e+04,3.000000,1.713395e+12,1.0,1.716042e+12,1.712670e+12,1.713408e+12,0.0,8.150000e+04,48059.000000,29183.000000
75%,3.904707e+09,1.400000e+05,8.047188e+06,8.000000,2510.500000,1.000000e+05,8.000000,1.713478e+12,1.0,1.716088e+12,1.713283e+12,1.713484e+12,0.0,1.250000e+05,78201.000000,42077.000000
max,3.906267e+09,1.200000e+08,1.034730e+08,9975.000000,750000.000000,8.500000e+07,967.000000,1.713573e+12,1.0,1.729125e+12,1.713562e+12,1.713573e+12,0.0,5.356000e+08,99901.000000,56045.000000


In [6]:
postings.isna().sum()

job_id                             0
company_name                    1719
title                              0
description                        7
max_salary                     94056
pay_period                     87776
location                           0
company_id                      1717
views                           1689
med_salary                    117569
min_salary                     94056
formatted_work_type                0
applies                       100529
original_listed_time               0
remote_allowed                108603
job_posting_url                    0
application_url                36665
application_type                   0
expiry                             0
closed_time                   122776
formatted_experience_level     29409
skills_desc                   121410
listed_time                        0
posting_domain                 39968
sponsored                          0
work_type                          0
currency                       87776
c

In [7]:
postings = postings.drop(['max_salary', 'med_salary', 'min_salary', 'pay_period', 'views', 'applies', 'remote_allowed', 'job_posting_url', 'expiry', 'closed_time', 'compensation_type', 'zip_code', 'fips'], axis= 1)

In [8]:
print(len(postings))
postings.isna().sum()

123849


job_id                             0
company_name                    1719
title                              0
description                        7
location                           0
company_id                      1717
formatted_work_type                0
original_listed_time               0
application_url                36665
application_type                   0
formatted_experience_level     29409
skills_desc                   121410
listed_time                        0
posting_domain                 39968
sponsored                          0
work_type                          0
currency                       87776
normalized_salary              87776
dtype: int64

In [9]:
postings = postings.dropna(subset= ['company_name', 'description'])
print(len(postings))
postings.isna().sum()

122124


job_id                             0
company_name                       0
title                              0
description                        0
location                           0
company_id                         0
formatted_work_type                0
original_listed_time               0
application_url                35055
application_type                   0
formatted_experience_level     27942
skills_desc                   119688
listed_time                        0
posting_domain                 38244
sponsored                          0
work_type                          0
currency                       86562
normalized_salary              86562
dtype: int64

In [10]:
postings.fillna({'application_url': ''}, inplace = True)
print(len(postings))
postings.isna().sum()

122124


job_id                             0
company_name                       0
title                              0
description                        0
location                           0
company_id                         0
formatted_work_type                0
original_listed_time               0
application_url                    0
application_type                   0
formatted_experience_level     27942
skills_desc                   119688
listed_time                        0
posting_domain                 38244
sponsored                          0
work_type                          0
currency                       86562
normalized_salary              86562
dtype: int64

In [11]:
postings['formatted_experience_level'].unique()

array([nan, 'Entry level', 'Mid-Senior level', 'Associate', 'Director',
       'Internship', 'Executive'], dtype=object)

In [12]:
postings.fillna({'formatted_experience_level': "Undefined"}, inplace= True)
postings.isna().sum()

job_id                             0
company_name                       0
title                              0
description                        0
location                           0
company_id                         0
formatted_work_type                0
original_listed_time               0
application_url                    0
application_type                   0
formatted_experience_level         0
skills_desc                   119688
listed_time                        0
posting_domain                 38244
sponsored                          0
work_type                          0
currency                       86562
normalized_salary              86562
dtype: int64

In [13]:
postings.fillna({'normalized_salary': 0, 'currency': 'Undefined'}, inplace= True)
postings.isna().sum()

job_id                             0
company_name                       0
title                              0
description                        0
location                           0
company_id                         0
formatted_work_type                0
original_listed_time               0
application_url                    0
application_type                   0
formatted_experience_level         0
skills_desc                   119688
listed_time                        0
posting_domain                 38244
sponsored                          0
work_type                          0
currency                           0
normalized_salary                  0
dtype: int64

In [14]:
postings.drop('skills_desc', axis= 1, inplace= True)
postings.isna().sum()

job_id                            0
company_name                      0
title                             0
description                       0
location                          0
company_id                        0
formatted_work_type               0
original_listed_time              0
application_url                   0
application_type                  0
formatted_experience_level        0
listed_time                       0
posting_domain                38244
sponsored                         0
work_type                         0
currency                          0
normalized_salary                 0
dtype: int64

In [15]:
postings.drop('posting_domain', axis= 1, inplace= True)
postings.isna().sum()

job_id                        0
company_name                  0
title                         0
description                   0
location                      0
company_id                    0
formatted_work_type           0
original_listed_time          0
application_url               0
application_type              0
formatted_experience_level    0
listed_time                   0
sponsored                     0
work_type                     0
currency                      0
normalized_salary             0
dtype: int64

In [16]:
postings.drop('original_listed_time', axis= 1, inplace= True)
postings.isna().sum()

job_id                        0
company_name                  0
title                         0
description                   0
location                      0
company_id                    0
formatted_work_type           0
application_url               0
application_type              0
formatted_experience_level    0
listed_time                   0
sponsored                     0
work_type                     0
currency                      0
normalized_salary             0
dtype: int64

In [17]:
postings.drop(['application_url', 'listed_time', 'sponsored'], axis= 1, inplace= True)
postings.isna().sum()

job_id                        0
company_name                  0
title                         0
description                   0
location                      0
company_id                    0
formatted_work_type           0
application_type              0
formatted_experience_level    0
work_type                     0
currency                      0
normalized_salary             0
dtype: int64

In [18]:
postings['formatted_work_type'].value_counts()

formatted_work_type
Full-time     97546
Contract      11955
Part-time      9488
Temporary      1176
Internship      955
Volunteer       555
Other           449
Name: count, dtype: int64

In [19]:
indices = postings[postings['formatted_work_type'] == 'Other'].index
postings = postings.drop(indices)
postings['formatted_work_type'].value_counts()

formatted_work_type
Full-time     97546
Contract      11955
Part-time      9488
Temporary      1176
Internship      955
Volunteer       555
Name: count, dtype: int64

In [20]:
postings = postings.drop('formatted_work_type', axis = 1)
postings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121675 entries, 0 to 123848
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      121675 non-null  int64  
 1   company_name                121675 non-null  object 
 2   title                       121675 non-null  object 
 3   description                 121675 non-null  object 
 4   location                    121675 non-null  object 
 5   company_id                  121675 non-null  float64
 6   application_type            121675 non-null  object 
 7   formatted_experience_level  121675 non-null  object 
 8   work_type                   121675 non-null  object 
 9   currency                    121675 non-null  object 
 10  normalized_salary           121675 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 11.1+ MB


In [21]:
postings = postings.drop('company_name', axis = 1)
postings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121675 entries, 0 to 123848
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      121675 non-null  int64  
 1   title                       121675 non-null  object 
 2   description                 121675 non-null  object 
 3   location                    121675 non-null  object 
 4   company_id                  121675 non-null  float64
 5   application_type            121675 non-null  object 
 6   formatted_experience_level  121675 non-null  object 
 7   work_type                   121675 non-null  object 
 8   currency                    121675 non-null  object 
 9   normalized_salary           121675 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 10.2+ MB


In [22]:
companies = pd.read_csv('/kaggle/input/linkedin-job-postings/companies/companies.csv')
companies.columns

Index(['company_id', 'name', 'description', 'company_size', 'state', 'country',
       'city', 'zip_code', 'address', 'url'],
      dtype='object')

In [23]:
companies.drop(['zip_code', 'address', 'url', 'state'], axis = 1, inplace = True)
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24473 entries, 0 to 24472
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company_id    24473 non-null  int64  
 1   name          24472 non-null  object 
 2   description   24176 non-null  object 
 3   company_size  21699 non-null  float64
 4   country       24473 non-null  object 
 5   city          24472 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ MB


# Preprocessing Text Data

In [24]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [26]:
stop_words = set(stopwords.words('english'))
english_words = set(words.words())
lemmatizer = WordNetLemmatizer()

In [27]:
def lemmatize_words(text):
    #word_tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text]
    return lemmas

In [28]:
URL_PATTERN = r'(?:http[s]?:\/\/.)?(?:www\.)?[-a-zA-Z0-9@%._\+~#=]{2,256}\.[a-z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&\/\/=]*)'

# Extract English Words

In [29]:
def preprocess_text(txt):

    if not txt:
        return ""

    txt = txt.lower()

    txt = re.sub(URL_PATTERN, '', txt) # to remove links
    
    txt = re.sub(r'<.*?>', '', txt) # to remove html tags </>
    
    txt = re.sub(r'[^a-z ]', '', txt) # remove non-alpha character
    
    txt = re.sub(r'\s+', ' ', txt) # replace multiple spaces with single space
    
    tokenized_txt = word_tokenize(txt)
    words = [word for word in tokenized_txt if ((len(word) > 1) and word not in stop_words)]
    lemmas = lemmatize_words(words)

    return lemmas

In [30]:
def clean_text(txt):

    tokenized_txt = preprocess_text(txt)
    txt = ' '.join(tokenized_txt) # remove stop words

    return txt

In [31]:
from tqdm import tqdm

In [33]:
preprocessed_tokens = postings['description'].apply(preprocess_text)

In [34]:
preprocessed_description = preprocessed_tokens.apply(' '.join)

In [32]:
samples_df = postings.sample(1000)
jobs = samples_df.to_dict(orient='records')

# Text Embedding

In [33]:
!pip install sentence_transformers

In [36]:
import torch

In [37]:
torch.set_default_device('cpu')

In [38]:
from sentence_transformers import SentenceTransformer

minilm_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
minilm_model.save('/kaggle/working/minilm')

In [38]:
!zip -r /kaggle/working/MiniLM.zip /kaggle/working/minilm

  adding: kaggle/working/minilm/ (stored 0%)
  adding: kaggle/working/minilm/config.json (deflated 47%)
  adding: kaggle/working/minilm/tokenizer.json (deflated 71%)
  adding: kaggle/working/minilm/1_Pooling/ (stored 0%)
  adding: kaggle/working/minilm/1_Pooling/config.json (deflated 57%)
  adding: kaggle/working/minilm/README.md (deflated 64%)
  adding: kaggle/working/minilm/model.safetensors (deflated 9%)
  adding: kaggle/working/minilm/2_Normalize/ (stored 0%)
  adding: kaggle/working/minilm/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/minilm/config_sentence_transformers.json (deflated 34%)
  adding: kaggle/working/minilm/vocab.txt (deflated 53%)
  adding: kaggle/working/minilm/tokenizer_config.json (deflated 73%)
  adding: kaggle/working/minilm/sentence_bert_config.json (deflated 4%)
  adding: kaggle/working/minilm/modules.json (deflated 62%)


# TF-IDF Vectorizer

In [39]:
%pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 99.8 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In [37]:
tfidf = TfidfVectorizer(min_df= 0.0001, max_df= 0.95, stop_words='english')
tfidf.fit(preprocessed_description)
print(len(tfidf.get_feature_names_out()))
tfidf

NameError: name 'preprocessed_description' is not defined

In [42]:
with open('/kaggle/working/tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [41]:
with open("/kaggle/input/job-postings-tfidf-vectorizer/scikitlearn/default/1/tfidf.model", 'rb') as f:
    tfidf = pickle.load(f)

# Encoding

In [42]:
postings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121675 entries, 0 to 123848
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      121675 non-null  int64  
 1   title                       121675 non-null  object 
 2   description                 121675 non-null  object 
 3   location                    121675 non-null  object 
 4   company_id                  121675 non-null  float64
 5   application_type            121675 non-null  object 
 6   formatted_experience_level  121675 non-null  object 
 7   work_type                   121675 non-null  object 
 8   currency                    121675 non-null  object 
 9   normalized_salary           121675 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 10.2+ MB


In [43]:
postings['work_type'].unique()

array(['FULL_TIME', 'INTERNSHIP', 'CONTRACT', 'PART_TIME', 'TEMPORARY',
       'VOLUNTEER'], dtype=object)

In [44]:
from sklearn.preprocessing import OneHotEncoder

wtonehotenc = OneHotEncoder(dtype= np.int32)
wtonehotenc.fit(postings[['work_type']].values)
wtonehotenc.categories_

[array(['CONTRACT', 'FULL_TIME', 'INTERNSHIP', 'PART_TIME', 'TEMPORARY',
        'VOLUNTEER'], dtype=object)]

In [45]:
np.sum(wtonehotenc.transform([[option] for option in ['CONTRACT', 'FULL_TIME', 'INTERNSHIP', 'PART_TIME', 'TEMPORARY',
        'VOLUNTEER']]).toarray(), axis=0, keepdims= True)

array([[1, 1, 1, 1, 1, 1]])

In [2]:
import numpy as np

In [5]:
np.array([[[0,1,0,1,0,0]]]).squeeze()

array([0, 1, 0, 1, 0, 0])

In [45]:
np.dot(np.sum(wtonehotenc.transform([[option] for option in ['CONTRACT', 'FULL_TIME', 'INTERNSHIP', 'PART_TIME', 'TEMPORARY',
        'VOLUNTEER']]).toarray(), axis=0, keepdims= True), np.array([[0,1,0,1,0,0]]).T).squeeze() 

array(2)

In [46]:
with open('/kaggle/working/work_type_onehot_enc.pkl', 'wb') as f:
    pickle.dump(wtonehotenc, f)

# Content Based Recommender

In [47]:
postings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 121675 entries, 0 to 123848
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      121675 non-null  int64  
 1   title                       121675 non-null  object 
 2   description                 121675 non-null  object 
 3   location                    121675 non-null  object 
 4   company_id                  121675 non-null  float64
 5   application_type            121675 non-null  object 
 6   formatted_experience_level  121675 non-null  object 
 7   work_type                   121675 non-null  object 
 8   currency                    121675 non-null  object 
 9   normalized_salary           121675 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 10.2+ MB


In [84]:
def get_matches_score(ind1, ind1_param, ind2, ind2_param, encoder):
    n_categs = len(encoder.categories_)
    
    ind1_enc = np.zeros(shape= (1, n_categs), dtype= np.int32)
    for option in ind1[ind1_param]:
        ind1_enc = ind1_enc + encoder.transform([[option]]).toarray()

    ind2_enc = encoder.transform([[ind[ind2_param]] for ind in ind2]).toarray()

    # Calculate similarity
    scores = np.dot(ind1_enc, ind2_enc.T)
    return scores


user_job_map = {
    'about': 'description',
    'preferred_work_types':'work_type',
    'experience_level': 'formatted_experience_level',
    'Expected Salary': 'normalized_salary'
}

def rename_key(dictionary, old_key, new_key):
    if old_key in dictionary:
        dictionary[new_key] = dictionary.pop(old_key)


def normalize(array: np.array):
    if np.any(array):
        return (array - np.min(array)) / (np.max(array) - np.min(array))
    return array


def user_job_recommend(user: dict, jobs: list[dict], weights, title_vectorizer, content_vectorizer, encoders):
    # Map user keys to job keys
    for key, value in user_job_map.items():
        rename_key(user, key, value)

    return job_recommend(user, jobs, weights, title_vectorizer, content_vectorizer, encoders)


def job_recommend(base_job: dict, jobs: list[dict], weights, title_vectorizer, content_vectorizer, encoders):
    similarity = dict()

    if weights['title']:
        # Get Title Embeddings
        base_job_title = title_vectorizer.encode([base_job['title']])
        jobs_title  = title_vectorizer.encode([job['title'] for job in jobs])
    
        # Calculate Title Similarity
        similarity['title'] = np.dot(base_job_title, jobs_title.T)[0]

    if weights['content']:
        # Get Description (Content) Embeddings
        for j in jobs:
            j['description'] = clean_text(j['description'])
        print("Started Vectorizing Descriptions")
        base_job_content_embd = title_vectorizer.encode([clean_text(base_job['description'])])
        jobs_content_embd = title_vectorizer.encode([job['description'] for job in jobs])

        # Calculate Content Embeddings Similarity
        similarity['content'] = np.dot(base_job_content_embd, jobs_content_embd.T)[0]

    # Calculate Work Type Similarity
    if 'work_type' in base_job.keys() and weights['work_type']:
        similarity['work_type'] = get_matches_score(base_job, 'work_type', jobs, 'work_type', encoders['work_type'])[0]
        
    else:
        similarity['work_type'] = 0

    # Get Skill Match
    if 'skills' in jobs[0] and weights['skills']:
        base_job_skill_embd = content_vectorizer.transform(base_job['skills']).toarray()
        jobs_skill_embd = content_vectorizer.transform([job['skills'] for job in jobs]).toarray()

        temp_similarity = np.dot(base_job_skill_embd, jobs_skill_embd.T)
        similarity['skills'] = np.average(temp_similarity)
    
    # Accumulate Scores
    n_jobs = len(jobs)
    n_features = len(similarity.keys())
    job_scores = np.zeros((n_jobs, ), dtype= np.float64)
    
    for feature, scores in similarity.items():
        job_scores += weights[feature] * normalize(scores)

    job_scores = sorted(enumerate(job_scores), key= lambda x: x[1], reverse= True)
    return job_scores

In [52]:
#Sample
user = {'title': 'Machine Learning Engineer',
             'about': '''Machine Learning Engineer with a strong focus on building intelligent systems that bridge the gap between user needs and actionable insights. Currently, I am working on developing a sophisticated recommender system designed to match user profiles with job descriptions, leveraging advanced natural language processing (NLP) and collaborative filtering techniques. My role involves end-to-end ownership of the project, from data preprocessing and feature engineering to model training, evaluation, and deployment.

With a solid foundation in Computer Science, I bring 4 years of experience in designing and implementing machine learning solutions that drive real-world impact. My expertise includes working with large-scale datasets, optimizing recommendation algorithms, and deploying models into production environments using cloud platforms like AWS and GCP. I am proficient in Python, TensorFlow, PyTorch, and Scikit-learn, and have hands-on experience with tools like Spark and Docker for scalable data processing and deployment.

In my current project, I am focused on improving the accuracy and personalization of job recommendations by incorporating user behavior data, contextual information, and advanced NLP techniques to better understand both user profiles and job descriptions. This involves experimenting with deep learning architectures, such as BERT-based models, and continuously iterating to enhance system performance. For example, I recently implemented a hybrid recommendation approach that combines content-based filtering with matrix factorization, resulting in a 15% improvement in recommendation relevance based on user feedback.

I am passionate about creating ethical, transparent, and user-centric AI systems that deliver meaningful value. Outside of work, I enjoy staying updated with the latest advancements in AI/ML, contributing to open-source projects like Hugging Face Transformers, and participating in technical communities such as Kaggle and Meetup groups. I am always open to connecting with professionals who share a passion for innovation and problem-solving. Let’s connect and explore how we can collaborate to build the future of intelligent systems!''',
            'preferred_work_types': ['FULL_TIME', 'INTERNSHIP'],
            'experience_level': 'Entry level',
            'expected_salary': None,
            'skills': ['python', 'scikitlearn', 'tensorflow', 'pandas']
            }

In [24]:
samples_df = postings.sample(2000)
test_samples = samples_df.to_dict(orient='records')
samples_df.head()

,job_id,title,description,location,company_id,application_type,formatted_experience_level,work_type,currency,normalized_salary
25766,3891077143,Senior Internal Auditor,Culbertson Resources is excited to present thi...,"Houston, TX",79422398.0,ComplexOnsiteApply,Mid-Senior level,FULL_TIME,Undefined,0.0
43761,3900082093,Workday Developer,"As a member of Workday HR technology team, gat...","Charlotte, NC",16388.0,ComplexOnsiteApply,Undefined,CONTRACT,Undefined,0.0
41714,3899526567,ETL Informatica Developer,Role- Etl Informatica DeveloperLocation - Phoe...,"Phoenix, AZ",60561.0,SimpleOnsiteApply,Undefined,FULL_TIME,Undefined,0.0
123681,3906260516,Emergency Room Registered Nurse,Job Description\n\nFacility: Gunnison Valley H...,"Gunnison, CO",67997480.0,SimpleOnsiteApply,Mid-Senior level,CONTRACT,Undefined,0.0
100503,3905213114,Entry Level Hospital Services Technician,"$20.19/hr, Daylight Shift, NO WEEKENDS and Gre...","Boston, MA",15798.0,OffsiteApply,Entry level,FULL_TIME,USD,41995.2


In [25]:
with open('/kaggle/working/test_data.pkl', 'wb') as f:
    pickle.dump(test_samples, f)

In [82]:
torch.set_default_device('cpu')

In [85]:
weights = {
    'title': 0.2,
    'content': 0.5,
    'work_type': 0.1,
    'skills': 0.2
}

encoders = {'work_type': wtonehotenc}

for idx, score in user_job_recommend(user, test_samples, weights,
                                    title_vectorizer= minilm_model,
                                    content_vectorizer= tfidf,
                                    encoders = encoders)[:5]:
    job = test_samples[idx]
    print(idx)
    print(job['title'])
    print(job['description'][:50])
    print(score, '\n\n')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Started Vectorizing Descriptions


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

1168
AI Engineer
lasalle network partner global distribution compan
0.7632841676473617 


438
Data Engineer
seek drive enthusiastic data engineer join dynamic
0.7509095728397369 


1546
Junior Software Engineer, AI
company overviewconveyor cuttingedge software comp
0.7383100003004074 


11
Software Engineer
summarythe software engineer responsible analysis 
0.7157655715942383 


668
Quantitative Developer
companyour client lead asset management firm billi
0.7131343692541122 


